In [130]:
# Import libraries
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
filepathBase = "/content/drive/Shareddrives/ML_Project/"

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
numeric_features = ['TUMOR_SIZE','TMB_NONSYNONYMOUS','LYMPH_NODES_EXAMINED_POSITIVE','NPI','AGE_AT_DIAGNOSIS']

categorical_features = ['CANCER_TYPE_DETAILED','ONCOTREE_CODE','COHORT','INTCLUST',
                      'CLAUDIN_SUBTYPE','THREEGENE','HISTOLOGICAL_SUBTYPE','HER2_SNP6']
genesToUse = ["TP53", "GATA3", "NCOR2", "MYO3A", "THSD7A",
              "ATR", "COL22A1", "PIK3R1", "RUNX1", "DNAH2",
              "MUC16", "CBFB", "BRIP1", "NDFIP1", "GLDC",
              "MAP3K1", "SETDB1", "UTRN", "TAF4B", "CDKN1B"]

binary_features = ['ER_STATUS','HER2_STATUS','PR_STATUS','CHEMOTHERAPY','ER_IHC','HORMONE_THERAPY',
                   'INFERRED_MENOPAUSAL_STATE','LATERALITY','RADIO_THERAPY','BREAST_SURGERY']

categorical_features = categorical_features + binary_features + genesToUse

ordinal_features = ['GRADE','TUMOR_STAGE','CELLULARITY']

drop_features = ["PATIENT_ID", "SAMPLE_ID", "CANCER_TYPE", "SAMPLE_TYPE", "SEX", 'OS_MONTHS', "VITAL_STATUS", "OS_STATUS", "RFS_STATUS"]

In [132]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop="if_binary", handle_unknown='ignore'))
])
#binary_transformer = Pipeline(steps=[
#    ('imputer', SimpleImputer(strategy='most_frequent')),
#    ('onehot', OneHotEncoder(drop='if_binary', handle_unknown = 'ignore'))
#])

#create a list of lists for categories to use
#as input for ordinal transformer
ordCat = [[1.0,2.0,3.0],[0.0,1.0,2.0,3.0,4.0],['Low','Moderate','High']] # These have to be exact matches for data (ie 1.0 instead of 1)

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories = ordCat))
])

In [133]:
mainDataPath = os.path.join(filepathBase, "Combined_data.csv")
mainDataframe = pd.read_csv(mainDataPath, header=0, sep=",")

In [134]:
print(mainDataframe.columns)

Index(['PATIENT_ID', 'SAMPLE_ID', 'CANCER_TYPE', 'CANCER_TYPE_DETAILED',
       'ER_STATUS', 'HER2_STATUS', 'GRADE', 'ONCOTREE_CODE', 'PR_STATUS',
       'SAMPLE_TYPE',
       ...
       'RASGEF1B', 'NT5E', 'CLRN2', 'LDLRAP1', 'CCND3', 'SMARCB1', 'PPP2CB',
       'SMARCD1', 'NRAS', 'STMN2'],
      dtype='object', length=209)


In [135]:
preprocessor = make_column_transformer(
    (numerical_transformer, numeric_features),
    (categorical_transformer, categorical_features),
    #(binary_transformer, binary_features),
    (ordinal_transformer, ordinal_features),
    ("drop", drop_features),
    #remainder="passthrough",
    remainder="drop",
    sparse_threshold=0
)
preprocessor

# mainDataframe.drop(["RFS_MONTHS"])
#mainDataframePath = os.path.join(filepathBase, "Combined_data.csv")
#mainDataframe = pd.read_csv("./Combined_data.csv", header=0)
#mainDataframe['RFS_STATUS'] = mainDataframe['RFS_STATUS'].map({'0:Not Recurred': 0, '1:Recurred': 1})

#just in case, drop rows with missing RFS_MONTHS
#mainDataframe.dropna(subset=['RFS_MONTHS'], inplace=True)

ColumnTransformer(sparse_threshold=0,
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['TUMOR_SIZE', 'TMB_NONSYNONYMOUS',
                                  'LYMPH_NODES_EXAMINED_POSITIVE', 'NPI',
                                  'AGE_AT_DIAGNOSIS']),
                                ('pipeline-2',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEnc...
                                ('pipeline-3',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal',
                                                  OrdinalEncoder(categories=[[1.0,
                                                                              2.0,
                                                                              3.0],
                                                                             [0.0,
                                                                              1.0,
                                                                              2.0,
                                                                              3.0,
                                                                              4.0],
                                                                             ['Low',
                                                                              'Moderate',
                                                                              'High']]))]),
                                 ['GRADE', 'TUMOR_STAGE', 'CELLULARITY']),
                                ('drop', 'drop',
                                 ['PATIENT_ID', 'SAMPLE_ID', 'CANCER_TYPE',
                                  'SAMPLE_TYPE', 'SEX', 'OS_MONTHS',
                                  'VITAL_STATUS', 'OS_STATUS', 'RFS_STATUS'])])

In [136]:
train_df, test_df = train_test_split(mainDataframe, test_size=0.3, random_state=123)
#print(train_df.columns)
#train_df.columns = train_df.columns.str.strip()
#test_df.columns = test_df.columns.str.strip()

X_train = train_df.drop(columns=["RFS_MONTHS"])
y_train = train_df["RFS_MONTHS"]

# test_df.columns
X_test = test_df.drop(columns=["RFS_MONTHS"])
y_test = test_df["RFS_MONTHS"]

In [137]:
from sklearn.dummy import DummyRegressor
dummy = DummyRegressor(strategy="median")
dummy_pipe = make_pipeline(preprocessor, dummy)
dummy_pipe.fit(X_train, y_train)
y_pred_dummy = dummy_pipe.predict(X_test)
print(r2_score(y_test, y_pred_dummy))

-0.01335560026097693


In [138]:
regSV = SVR()

pipeSV = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', regSV)
    ])

In [139]:
pipeSV.fit(X_train, y_train)
print(cross_val_score(pipeSV, X_train, y_train))

SVpred = pipeSV.predict(X_test)
print(type(y_test))
print(type(SVpred))
#print(RFpred)

print(r2_score(y_test, SVpred))

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[0.01617056 0.06255396 0.03145037 0.03457495 0.07171627]
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
0.06281351223077081


In [140]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, loguniform
# SVR regresser hyperparamater search
# This code was adapted from demo 8

paramGrid = {
    "regressor__kernel": ['linear', 'rbf', 'poly'],
    "regressor__C": loguniform(1e0, 1e3),
    "regressor__gamma": loguniform(1e-4, 1e-1),
    "regressor__epsilon": uniform(0.01, 0.5)
}

scoring = [
    "neg_mean_squared_error",
    "neg_mean_absolute_error",
    "r2"
]
# Try 30 different combinations of the defined values
random_search = RandomizedSearchCV(
    pipeSV, param_distributions=paramGrid, n_jobs=-1, n_iter=50, cv=3, scoring=scoring, refit="r2", random_state=123
)
random_search.fit(X_train, y_train)

# Show the fit time and average cross validation score for each paramater combination, ordered with the best score first
print(pd.DataFrame(random_search.cv_results_)[
    [
       "mean_test_neg_mean_squared_error",
        "param_regressor__kernel",
        "param_regressor__C",
        "param_regressor__gamma",
        "param_regressor__epsilon",
        "mean_fit_time",
        "rank_test_neg_mean_squared_error",
        "mean_test_neg_mean_absolute_error",
        "mean_test_r2"
    ]
].set_index("rank_test_neg_mean_squared_error").sort_index().T)
# Rank on f1
print(pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_neg_mean_squared_error",
        "param_regressor__kernel",
        "param_regressor__C",
        "param_regressor__gamma",
        "param_regressor__epsilon",
        "mean_fit_time",
        "rank_test_r2",
        "mean_test_neg_mean_absolute_error",
        "mean_test_r2"
    ]
].set_index("rank_test_r2").sort_index().T)
print(random_search.best_score_)


rank_test_neg_mean_squared_error           1            2            3   \
mean_test_neg_mean_squared_error  -5404.23992 -5404.753862 -5435.690361   
param_regressor__kernel                   rbf          rbf         poly   
param_regressor__C                 110.637486     8.950689     8.645372   
param_regressor__gamma               0.003398     0.039713     0.047881   
param_regressor__epsilon             0.447728     0.217413     0.223176   
mean_fit_time                        0.218573       0.2283     0.268171   
mean_test_neg_mean_absolute_error  -60.222686   -60.100856   -60.445917   
mean_test_r2                         0.091996     0.092026     0.087443   

rank_test_neg_mean_squared_error            4            5            6   \
mean_test_neg_mean_squared_error  -5470.451654 -5477.435294 -5506.109843   
param_regressor__kernel                 linear          rbf          rbf   
param_regressor__C                    1.346111    20.636838   143.698068   
param_regressor__gam

In [141]:
params = random_search.best_params_
#print(params)
pipeSV =Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', SVR(kernel=params['regressor__kernel'], C=params['regressor__C'],
              gamma=params['regressor__gamma'], epsilon=params['regressor__epsilon'])
        )
    ])
pipeSV.fit(X_train,y_train)
SVpred = pipeSV.predict(X_test)
print(r2_score(y_test, SVpred))

0.10866428367262593


In [142]:
regRF = RandomForestRegressor()

pipeRF = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', regRF)
    ])

In [143]:
pipeRF.fit(X_train, y_train)
print(cross_val_score(pipeRF, X_train, y_train))

RFpred = pipeRF.predict(X_test)
print(type(y_test))
print(type(RFpred))
#print(RFpred)

print(r2_score(y_test, RFpred))

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[0.05539136 0.08099943 0.12199979 0.05764652 0.15396365]
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
0.14720249205408675


In [144]:
paramGrid = {
    "regressor__max_depth": [None]+list(range(10,65,1)),
    "regressor__max_features": [None,"sqrt","log2"],
    "regressor__n_estimators": list(range(400,2000,100))
}

scoring = [
    "neg_mean_squared_error",
    "neg_mean_absolute_error",
    "r2"
]
# Try 30 different combinations of the defined values
random_search = RandomizedSearchCV(
    pipeRF, param_distributions=paramGrid, n_jobs=-1, n_iter=50, cv=3, scoring=scoring, refit="r2", random_state=123
)
random_search.fit(X_train, y_train)

# Show the fit time and average cross validation score for each paramater combination, ordered with the best score first
print(pd.DataFrame(random_search.cv_results_)[
    [
       "mean_test_neg_mean_squared_error",
        "param_regressor__max_depth",
        "param_regressor__max_features",
        "param_regressor__n_estimators",
        "mean_fit_time",
        "rank_test_neg_mean_squared_error",
        "mean_test_neg_mean_absolute_error",
        "mean_test_r2"
    ]
].set_index("rank_test_neg_mean_squared_error").sort_index().T)
# Rank on f1
print(pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_neg_mean_squared_error",
        "param_regressor__max_depth",
        "param_regressor__max_features",
        "param_regressor__n_estimators",
        "mean_fit_time",
        "rank_test_r2",
        "mean_test_neg_mean_absolute_error",
        "mean_test_r2"
    ]
].set_index("rank_test_r2").sort_index().T)
print(random_search.best_score_)

rank_test_neg_mean_squared_error            1            2            3   \
mean_test_neg_mean_squared_error  -5163.465383 -5166.344954 -5166.567885   
param_regressor__max_depth                  16           13           14   
param_regressor__max_features             sqrt         sqrt         sqrt   
param_regressor__n_estimators             1200          400         1100   
mean_fit_time                         7.324324     2.225923     6.342811   
mean_test_neg_mean_absolute_error   -60.016447   -60.033158   -60.067921   
mean_test_r2                          0.133061     0.132631     0.132448   

rank_test_neg_mean_squared_error            4            5            6   \
mean_test_neg_mean_squared_error  -5172.522086 -5176.892767 -5183.721669   
param_regressor__max_depth                  13           28           55   
param_regressor__max_features             sqrt         sqrt         sqrt   
param_regressor__n_estimators             1600         1000         1900   
mean_fit_ti

In [145]:
params = random_search.best_params_
#print(params)
pipeRF =Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=params['regressor__n_estimators'],
              max_depth=params['regressor__max_depth'],max_features=params['regressor__max_features'])
        )
    ])
pipeRF.fit(X_train,y_train)
RFpred = pipeRF.predict(X_test)
print(r2_score(y_test, RFpred))

0.1587658915265857


In [146]:
rf = pipeRF.named_steps['regressor']
feature_names = pipeRF.named_steps['preprocessor'].get_feature_names_out()

feat_imp = (
    pd.DataFrame({'feature': feature_names, 'importance': rf.feature_importances_})
      .sort_values('importance', ascending=False)
)

print(feat_imp)

                                              feature  importance
3                                     pipeline-1__NPI    0.090452
4                        pipeline-1__AGE_AT_DIAGNOSIS    0.086945
0                              pipeline-1__TUMOR_SIZE    0.072373
1                       pipeline-1__TMB_NONSYNONYMOUS    0.058675
2           pipeline-1__LYMPH_NODES_EXAMINED_POSITIVE    0.049364
..                                                ...         ...
18                      pipeline-2__ONCOTREE_CODE_MBC    0.000116
12  pipeline-2__CANCER_TYPE_DETAILED_Metaplastic B...    0.000114
51       pipeline-2__HISTOLOGICAL_SUBTYPE_Metaplastic    0.000071
20                      pipeline-2__ONCOTREE_CODE_PBS    0.000060
6   pipeline-2__CANCER_TYPE_DETAILED_Breast Angios...    0.000048

[93 rows x 2 columns]


In [147]:
from sklearn.ensemble import GradientBoostingRegressor

regGB = GradientBoostingRegressor()

pipeGB = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', regGB)
    ])

In [148]:
pipeGB.fit(X_train, y_train)
print(cross_val_score(pipeGB, X_train, y_train))

GBpred = pipeGB.predict(X_test)
print(type(y_test))
print(type(GBpred))
#print(RFpred)

print(r2_score(y_test, GBpred))

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[0.063908   0.10493053 0.11861943 0.04704471 0.16793182]
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
0.16198407238566048


In [149]:
paramGrid = {
    'regressor__n_estimators': np.arange(50, 500, 50),
    'regressor__learning_rate': np.linspace(0.01, 0.2, 10),
    'regressor__max_depth': np.arange(3, 10),
    'regressor__subsample': np.linspace(0.6, 1.0, 5),
    'regressor__min_samples_split': np.arange(2, 20),
    'regressor__min_samples_leaf': np.arange(1, 10)
}

scoring = [
    "neg_mean_squared_error",
    "neg_mean_absolute_error",
    "r2"
]
# Try 30 different combinations of the defined values
random_search = RandomizedSearchCV(
    pipeGB, param_distributions=paramGrid, n_jobs=-1, n_iter=50, cv=3, scoring=scoring, refit="r2", random_state=123
)
random_search.fit(X_train, y_train)

# Show the fit time and average cross validation score for each paramater combination, ordered with the best score first
print(pd.DataFrame(random_search.cv_results_)[
    [
       "mean_test_neg_mean_squared_error",
        "param_regressor__n_estimators",
        "param_regressor__learning_rate",
        "param_regressor__max_depth",
        "param_regressor__subsample",
        "param_regressor__min_samples_split",
        "param_regressor__min_samples_leaf",
        "mean_fit_time",
        "rank_test_neg_mean_squared_error",
        "mean_test_neg_mean_absolute_error",
        "mean_test_r2"
    ]
].set_index("rank_test_neg_mean_squared_error").sort_index().T)
# Rank on f1
print(pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_neg_mean_squared_error",
        "param_regressor__n_estimators",
        "param_regressor__learning_rate",
        "param_regressor__max_depth",
        "param_regressor__subsample",
        "param_regressor__min_samples_split",
        "param_regressor__min_samples_leaf",
        "mean_fit_time",
        "rank_test_r2",
        "mean_test_neg_mean_absolute_error",
        "mean_test_r2"
    ]
].set_index("rank_test_r2").sort_index().T)
print(random_search.best_score_)

rank_test_neg_mean_squared_error             1            2            3   \
mean_test_neg_mean_squared_error   -5153.015112 -5171.112506 -5174.287305   
param_regressor__n_estimators        350.000000   250.000000   200.000000   
param_regressor__learning_rate         0.010000     0.010000     0.010000   
param_regressor__max_depth             3.000000     7.000000     7.000000   
param_regressor__subsample             0.900000     0.800000     0.600000   
param_regressor__min_samples_split     5.000000     2.000000     8.000000   
param_regressor__min_samples_leaf      9.000000     9.000000     2.000000   
mean_fit_time                          1.879776     2.408643     1.671091   
mean_test_neg_mean_absolute_error    -59.751008   -59.648253   -59.797895   
mean_test_r2                           0.134608     0.131645     0.131156   

rank_test_neg_mean_squared_error             4            5            6   \
mean_test_neg_mean_squared_error   -5235.267934 -5259.852598 -5270.025437  

In [150]:
params = random_search.best_params_
#print(params)
pipeGB =Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', GradientBoostingRegressor(subsample=params['regressor__subsample'], n_estimators=params['regressor__n_estimators'],
              min_samples_split=params['regressor__min_samples_split'], min_samples_leaf=params['regressor__min_samples_leaf'],
              max_depth=params['regressor__max_depth'],learning_rate=params['regressor__learning_rate'])
        )
    ])
pipeGB.fit(X_train,y_train)
GBpred = pipeGB.predict(X_test)
print(r2_score(y_test, GBpred))

0.15087478336582394


In [151]:
GB = pipeGB.named_steps['regressor']
feature_names = pipeGB.named_steps['preprocessor'].get_feature_names_out()

feat_imp = (
    pd.DataFrame({'feature': feature_names, 'importance': GB.feature_importances_})
      .sort_values('importance', ascending=False)
)

print(feat_imp)

                                      feature  importance
3                             pipeline-1__NPI    0.229955
4                pipeline-1__AGE_AT_DIAGNOSIS    0.152173
21                     pipeline-2__COHORT_1.0    0.093622
2   pipeline-1__LYMPH_NODES_EXAMINED_POSITIVE    0.081175
22                     pipeline-2__COHORT_2.0    0.071155
..                                        ...         ...
78                        pipeline-2__RUNX1_1    0.000000
83                       pipeline-2__NDFIP1_1    0.000000
86                       pipeline-2__SETDB1_1    0.000000
87                         pipeline-2__UTRN_1    0.000000
89                       pipeline-2__CDKN1B_1    0.000000

[93 rows x 2 columns]
